**SA305 &#x25aa; Linear Programming &#x25aa; Spring 2021 &#x25aa; Uhan**

# Project. The Diet Problem

⚠️ In order to complete this project, you need to have Pyomo and GLPK installed on your computer.

## Introduction

In the diet problem, we are given a set of foods and a set of nutrients. We are also given the cost per serving for each food, and the minimum and maximum daily requirements for each nutrient. In addition, we have the amount of each nutrient in a serving of each food. The goal is to find the cheapest combination of foods that will satisfy all the daily nutritional requirements.

Some history about the diet problem [from NEOS](https://neos-guide.org/content/diet-problem):

> The diet problem was one of the first optimization problems studied in the 1930s and 1940s. The problem was motivated by the Army's desire to minimize the cost of feeding GIs in the field while still providing a healthy diet. One of the early researchers to study the problem was George Stigler, who made an educated guess of an optimal solution using a heuristic method. His guess for the cost of an optimal diet was \\$39.93 per year (1939 prices). In the fall of 1947, Jack Laderman of the Mathematical Tables Project of the National Bureau of Standards used the newly developed simplex method to solve Stigler's model. As the first "large scale" computation in optimization, the linear program consisted of nine equations in 77 unknowns. It took nine clerks using hand-operated desk calculators 120 man days to solve for the optimal solution of \\$39.69. Stigler's guess was off by only \\$0.24 per year!

## A parameterized model for the diet problem

__Sets.__
\begin{align*}
F & = \mbox{set of foods}\\
N & = \mbox{set of nutrients}\\
\end{align*}

__Parameters.__
\begin{align*}
c_j & = \mbox{cost of food $j$} && \text{for } j \in F\\
t_j & = \mbox{type of food $j$} && \text{for } j \in F\\
l_i & = \mbox{minimum daily requirement for nutrient $i$} && \text{for } i \in N\\
u_i & = \mbox{minimum daily requirement for nutrient $i$} && \text{for } i \in N\\
a_{ij} & = \mbox{amount of nutrient $i$ in food $j$} && \text{for } i \in N, j \in F
\end{align*}

__Decision variables.__
\begin{equation*}
x_j = \mbox{amount of food $j$ chosen} \quad \text{for } j \in F
\end{equation*}

__Objective function and constraints.__
\begin{array}{llll}
\min & \sum_{j \in M} c_j x_j & & \mbox{ (total cost) }\\
\mbox{s.t.}&  l_i \leq \sum_{j \in M} a_{ij}x_j \leq u_i & i \in N & \mbox{ (nutrition requirements) }\\
 & x_j \geq 0 & j \in F &
\end{array}

Note that the parameter $t_j$ is *not* used in the formulation above, but we will use it below.

---

## Reading and processing data

Reading and processing data is often a significant portion of the work necessary to get an optimization model up and running. In this project, you will learn one way to approach these tasks.

### Reading data with pandas

Typically, data is stored in a file such as a spreadsheet or comma separated value (CSV) file. There are many ways to read in such files using Python. 

For this project, you will use a library called [Pandas](https://pandas.pydata.org/). The main object in Pandas is the __DataFrame__, which is a two-dimensional table, with rows and columns.

In the same folder as this notebook, there is an Excel file called `mcdonalds.xlsx`. Before you go on, open this file and familiarize yourself with the three worksheets contained in that file. It's often useful to look at the actual raw data before processing it.

We start by importing Pandas. Then we use the Pandas method `read_excel` to read the `Food` worksheet in `mcdonalds.xlsx` into a Pandas DataFrame called `food_df`. Next, we use the DataFrame method `head` to display the first 5 rows of `food_df`.

In [1]:
import pandas as pd

food_df = pd.read_excel('mcdonalds.xlsx', sheet_name='Food', index_col='Food')

food_df.head()

,Cost,Type
Food,,
Hamburger,0.59,Main
Cheeseburger,0.69,Main
Quarter Pounder w/ Cheese,1.84,Main
McLean Deluxe,2.09,Main
McLean Deluxe w/ Cheese,2.19,Main


Pandas assigns a **label** to each row/observation of a DataFrame, based on the input data. These labels are called the **index**. In our code above, we used the keyword argument `index_col='Food'` to tell Python which column of the worksheet to use as the index. In the output above, you can see the labels in boldface on the left of each row.

### Processing data 

Pandas has handy functions to create the lists and dictionaries we use to formulate our optimization models in Pyomo. 

For example, here is code to create a list for the set of foods $F$ and a dictionary for the food costs $c_j$ for $j \in F$.

In [2]:
food_list = food_df.index.to_list()

food_list

['Hamburger',
 'Cheeseburger',
 'Quarter Pounder w/ Cheese',
 'McLean Deluxe',
 'McLean Deluxe w/ Cheese',
 'Big Mac',
 'Filet-O-Fish',
 'McGrilled Chicken',
 'McChicken Sandwich',
 'Fries, small',
 'Fries, large',
 'Fries, super',
 'Chicken McNuggets (6 pcs)',
 'Chicken McNuggets (9 pcs)',
 'Chicken McNuggets (20 pcs)',
 'Hot Mustard Sauce',
 'Barbeque Sauce',
 "Sweet 'N Sour Sauce",
 'Honey',
 'Chef Salad',
 'Chunky Chicken Salad',
 'Garden Salad',
 'Side Salad',
 'Croutons',
 'Bacon Bits',
 'Bleu Cheese Dressing',
 'Ranch Dressing',
 '1000 Island Dressing',
 'Lite Vinaigrette Dressing',
 'French Rdcd Cal Dressing',
 'Egg McMuffin',
 'Sausage McMuffin',
 'Sansage McMuffin with Egg',
 'English Muffin',
 'Sausage Biscuit',
 'Sausage Biscuit with Egg',
 'Bacon, Egg & Cheese Biscuit',
 'Hash Browns',
 'Breakfast Burrito',
 'Cheerios',
 'Wheaties',
 'Apple Danish',
 'Cheese Danish',
 'Cinnamon Raisin Danish',
 'Raspberry Danish',
 'Lowfat Frozen Yogurt Cone',
 'Vanilla Shake',
 'Chocolate

In [3]:
cost_dict = food_df.to_dict()['Cost']

cost_dict

{'Hamburger': 0.59,
 'Cheeseburger': 0.69,
 'Quarter Pounder w/ Cheese': 1.84,
 'McLean Deluxe': 2.09,
 'McLean Deluxe w/ Cheese': 2.19,
 'Big Mac': 1.84,
 'Filet-O-Fish': 1.44,
 'McGrilled Chicken': 2.29,
 'McChicken Sandwich': 2.04,
 'Fries, small': 0.77,
 'Fries, large': 1.17,
 'Fries, super': 1.49,
 'Chicken McNuggets (6 pcs)': 1.69,
 'Chicken McNuggets (9 pcs)': 2.34,
 'Chicken McNuggets (20 pcs)': 4.49,
 'Hot Mustard Sauce': 0.0,
 'Barbeque Sauce': 0.0,
 "Sweet 'N Sour Sauce": 0.0,
 'Honey': 0.0,
 'Chef Salad': 2.94,
 'Chunky Chicken Salad': 2.99,
 'Garden Salad': 1.99,
 'Side Salad': 1.39,
 'Croutons': 0.0,
 'Bacon Bits': 0.0,
 'Bleu Cheese Dressing': 0.0,
 'Ranch Dressing': 0.0,
 '1000 Island Dressing': 0.0,
 'Lite Vinaigrette Dressing': 0.0,
 'French Rdcd Cal Dressing': 0.0,
 'Egg McMuffin': 1.49,
 'Sausage McMuffin': 1.29,
 'Sansage McMuffin with Egg': 1.49,
 'English Muffin': 0.84,
 'Sausage Biscuit': 0.79,
 'Sausage Biscuit with Egg': 1.49,
 'Bacon, Egg & Cheese Biscuit': 1

Sometimes, however, we need to work harder to process the data. For example, consider the tabular data in the `FoodNutrients` worksheet of `mcdonalds.xlsx`. We can read it in as follows:

In [4]:
food_nut_df = pd.read_excel('mcdonalds.xlsx', sheet_name='FoodNutrients', index_col='Food')

food_nut_dict_raw = food_nut_df.to_dict()

food_nut_dict_raw

{'Cal': {'Hamburger': 255,
  'Cheeseburger': 305,
  'Quarter Pounder w/ Cheese': 510,
  'McLean Deluxe': 320,
  'McLean Deluxe w/ Cheese': 370,
  'Big Mac': 500,
  'Filet-O-Fish': 370,
  'McGrilled Chicken': 400,
  'McChicken Sandwich': 470,
  'Fries, small': 220,
  'Fries, large': 400,
  'Fries, super': 620,
  'Chicken McNuggets (6 pcs)': 270,
  'Chicken McNuggets (9 pcs)': 405,
  'Chicken McNuggets (20 pcs)': 900,
  'Hot Mustard Sauce': 70,
  'Barbeque Sauce': 50,
  "Sweet 'N Sour Sauce": 60,
  'Honey': 45,
  'Chef Salad': 170,
  'Chunky Chicken Salad': 150,
  'Garden Salad': 50,
  'Side Salad': 30,
  'Croutons': 50,
  'Bacon Bits': 15,
  'Bleu Cheese Dressing': 250,
  'Ranch Dressing': 220,
  '1000 Island Dressing': 225,
  'Lite Vinaigrette Dressing': 50,
  'French Rdcd Cal Dressing': 160,
  'Egg McMuffin': 280,
  'Sausage McMuffin': 345,
  'Sansage McMuffin with Egg': 430,
  'English Muffin': 170,
  'Sausage Biscuit': 420,
  'Sausage Biscuit with Egg': 505,
  'Bacon, Egg & Cheese B

If you look carefully, you'll see that `food_nut_dict_raw` is actually a dictionary of dictionaries, or a nested dictionary. The information in the nested dictionary `food_nut_dict_raw` gives us the information we need for the parameter $a_{ij}$ for $i \in N$ and $j \in F$. However, this dictionary isn't in the form that Python requires.

Let's illustrate this concretely. To get the dictionary that contains calorie information, we write

In [5]:
food_nut_dict_raw['Cal']

{'Hamburger': 255,
 'Cheeseburger': 305,
 'Quarter Pounder w/ Cheese': 510,
 'McLean Deluxe': 320,
 'McLean Deluxe w/ Cheese': 370,
 'Big Mac': 500,
 'Filet-O-Fish': 370,
 'McGrilled Chicken': 400,
 'McChicken Sandwich': 470,
 'Fries, small': 220,
 'Fries, large': 400,
 'Fries, super': 620,
 'Chicken McNuggets (6 pcs)': 270,
 'Chicken McNuggets (9 pcs)': 405,
 'Chicken McNuggets (20 pcs)': 900,
 'Hot Mustard Sauce': 70,
 'Barbeque Sauce': 50,
 "Sweet 'N Sour Sauce": 60,
 'Honey': 45,
 'Chef Salad': 170,
 'Chunky Chicken Salad': 150,
 'Garden Salad': 50,
 'Side Salad': 30,
 'Croutons': 50,
 'Bacon Bits': 15,
 'Bleu Cheese Dressing': 250,
 'Ranch Dressing': 220,
 '1000 Island Dressing': 225,
 'Lite Vinaigrette Dressing': 50,
 'French Rdcd Cal Dressing': 160,
 'Egg McMuffin': 280,
 'Sausage McMuffin': 345,
 'Sansage McMuffin with Egg': 430,
 'English Muffin': 170,
 'Sausage Biscuit': 420,
 'Sausage Biscuit with Egg': 505,
 'Bacon, Egg & Cheese Biscuit': 440,
 'Hash Browns': 130,
 'Breakfa

So, to get the calories in a Cheeseburger from `foot_nut_dict_raw`, we write

In [6]:
food_nut_dict_raw['Cal']['Cheeseburger']

305

However, to model the parameter $a_{ij}$, Pyomo needs a dictionary whose keys are _tuples_: in particular, of the form `(i, j)`, where `i` corresponds to a nutrient, and `j` corresponds to a food. In other words, we want a dictionary where we would write the following to get the calories in a Cheeseburger:

```python
food_nut_dict[('Cal','Cheeseburger')]
```

We can convert the "raw" nested dictionary into the form we need for Pyomo by using a __dictionary comprehension__, which is a short code block that creates a dictionary from other data structures using for statements and conditionals. Consider the following example.

In [7]:
dict_of_dicts = {
    'A': {'first': 1, 'second': 2, 'third': 3},
    'B': {'first': 11, 'second': 12, 'third': 13}
}

tuple_dict = {
    (i, j): dict_of_dicts[i][j] 
    for i in dict_of_dicts.keys() 
    for j in dict_of_dicts[i].keys()
}

tuple_dict

{('A', 'first'): 1,
 ('A', 'second'): 2,
 ('A', 'third'): 3,
 ('B', 'first'): 11,
 ('B', 'second'): 12,
 ('B', 'third'): 13}

---

## Your assignment

__1.__ Using Pandas, read in the data in `mcdonalds.xlsx` into lists and dictionaries that define values for the sets and parameters in the parameterized model above.

You will need:

- a list of foods
- a list of nutrients
- a dictionary that maps foods to their types
- a dictionary that maps nutrients to their minimum daily requirement
- a dictionary that maps nutrients to their maximum daily requirement
- a dictionary that maps (nutrient, food) to the amount of each nutrient in each food

In [8]:
# Solution
food_df = pd.read_excel('mcdonalds.xlsx',
                       sheet_name='Food',
                       index_col='Food')

nut_df = pd.read_excel('mcdonalds.xlsx',
                       sheet_name='NutritionRequirements',
                       index_col='Nutrient')

food_nut_df = pd.read_excel('mcdonalds.xlsx',
                            sheet_name='FoodNutrients',
                            index_col='Food')


food_list = food_df.index.to_list()
nutrient_list = nut_df.index.to_list()

food_type_dict = food_df.to_dict()['Type']
lower_bd_dict = nut_df.to_dict()['Lower']
upper_bd_dict = nut_df.to_dict()['Upper']

food_nut_dict_raw = food_nut_df.to_dict()
food_nut_dict = {(i,j): food_nut_dict_raw[i][j] 
                 for i in food_nut_dict_raw.keys() 
                 for j in food_nut_dict_raw[i].keys()}

__2.__ Implement the parameterized model above in Pyomo. Use your answer from the previous question and the code from Lab 2 as a template.

In [9]:
# Solution
import pyomo.environ as pyo

model = pyo.ConcreteModel()

model.F = pyo.Set(initialize=food_list)
model.N = pyo.Set(initialize=nutrient_list)

model.c = pyo.Param(model.F, initialize=cost_dict)
model.l = pyo.Param(model.N, initialize=lower_bd_dict)
model.u = pyo.Param(model.N, initialize=upper_bd_dict)
model.a = pyo.Param(model.N, model.F,initialize=food_nut_dict)

model.x = pyo.Var(model.F, domain=pyo.NonNegativeReals)

def total_cost_rule(model):
    return sum(model.c[j] * model.x[j] for j in model.F)

model.total_cost = pyo.Objective(
    rule=total_cost_rule,
    sense=pyo.minimize
)

def requirements_lower_rule(model, i):
    return sum(model.a[i,j] * model.x[j] for j in model.F) >= model.l[i]

model.requirements_lower = pyo.Constraint(model.N, rule=requirements_lower_rule)

def requirements_upper_rule(model, i):
    return sum(model.a[i,j] * model.x[j] for j in model.F) <= model.u[i]

model.requirements_upper = pyo.Constraint(model.N, rule=requirements_upper_rule)

__3.__ Solve the model. Output the solution.

In particular, write code that invokes `glpk` to solve the linear program. For the solution, only print decision variables that are positive in an optimal solution.

In [10]:
# Solution
result = pyo.SolverFactory('glpk').solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpqxf4sb38.glpk.raw
 --wglp /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpr38qmgrm.glpk.glp
 --cpxlp /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpav_tc37f.pyomo.lp
Reading problem data from '/var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpav_tc37f.pyomo.lp'...
25 rows, 64 columns, 1075 non-zeros
1273 lines were read
Writing problem data to '/var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpr38qmgrm.glpk.glp'...
1244 lines were written
GLPK Simplex Optimizer, v4.65
25 rows, 64 columns, 1075 non-zeros
Preprocessing...
19 rows, 63 columns, 834 non-zeros
Scaling...
 A: min|aij| =  5.000e-01  max|aij| =  1.933e+03  ratio =  3.866e+03
GM: min|aij| =  1.391e-01  max|aij| =  7.188e+00  ratio =  5.167e+01
EQ: min|aij| =  1.935e-02  max|aij| =  1.000e+00  ratio =  5.167e+01
Constructing initial basis...
Size of triangular part is 19
      0: 

In [11]:
# Solution
if result.solver.termination_condition == pyo.TerminationCondition.optimal:
    print(f'Optimal cost: {pyo.value(model.total_cost)}\n')
    print('Optimal diet:')
    
    for j in model.F:
        if pyo.value(model.x[j]) > 0:
            print(f'  {j}: {pyo.value(model.x[j])}')

Optimal cost: 5.363036218769084

Optimal diet:
  Cheeseburger: 2.05860944922915
  Sweet 'N Sour Sauce: 4.12271821458596
  Honey: 15.7682858968087
  Chunky Chicken Salad: 0.0362300014087301
  Cheerios: 2.26957075755894
  1% Lowfat Milk: 1.77757979894407
  Orange Juice: 0.408177763171085


__4.__ Interpret and critique the solution. Ignore integrality issues and focus on the composition of the menu.

*Write your answer here. Double-click to edit.*

*Solution.* The most glaring problem is that the diet consists of eating at least 16 packs of honey and 4 packs of sweet 'n sour sauce, which seems really unusual.

__5.__ Let's create a more palatable diet, using the food types.

For example, suppose we want to ensure that there are at most 6 condiments in our chosen diet. We can write such a constraint like this:

\begin{equation*}
\sum_{j \in F: t_j = \text{Condiment}} x_j \le 6
\end{equation*}

Recall from Lab 3 that we can write such a constraint with a sum using ":" notation in Pyomo like this:

```python
def condiment_rule(model):
    return sum(model.x[j] for j in model.F if model.food_type[j] == 'Condiment') <= 6

model.condiment = pyo.Constraint(rule=condiment_rule)
```

Do the following to create a more palatable diet:

- Add the food types $t_j$ parameter to your model.
- Add constraints using this parameter to enforce the following:
    - There are no more than 6 condiments in the diet.
    - There are at least 2 main dishes in the diet.
    - There is at least 1 breakfast dish in the diet.
    - There are at least 3 beverages in the diet.

⚠️ Use the keyword argument `within=pyo.Any` in `pyo.Param`, to let Pyomo know that this parameter is not numeric.

In [12]:
# Solution
model.food_type = pyo.Param(model.F, initialize=food_type_dict, within=pyo.Any)

In [13]:
def condiment_rule(model):
    return sum(model.x[j] for j in model.F if model.food_type[j] == 'Condiment') <= 6

model.condiment = pyo.Constraint(rule=condiment_rule)

def mains_rule(model):
    return sum(model.x[j] for j in model.F if model.food_type[j] == 'Main') >= 2

model.mains = pyo.Constraint(rule=mains_rule)

def breakfast_rule(model):
    return sum(model.x[j] for j in model.F if model.food_type[j] == 'Breakfast') >= 1

model.breakfast = pyo.Constraint(rule=breakfast_rule)

def beverage_rule(model):
    return sum(model.x[j] for j in model.F if model.food_type[j] == 'Beverage') >= 3

model.beverage = pyo.Constraint(rule=beverage_rule)

__6__. Solve the new model. Output the solution.

In [14]:
# Solution
result = pyo.SolverFactory('glpk').solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpta5x9w3x.glpk.raw
 --wglp /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpwdfobr_y.glpk.glp
 --cpxlp /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmp_esztqz9.pyomo.lp
Reading problem data from '/var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmp_esztqz9.pyomo.lp'...
29 rows, 64 columns, 1132 non-zeros
1342 lines were read
Writing problem data to '/var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpwdfobr_y.glpk.glp'...
1309 lines were written
GLPK Simplex Optimizer, v4.65
29 rows, 64 columns, 1132 non-zeros
Preprocessing...
23 rows, 63 columns, 891 non-zeros
Scaling...
 A: min|aij| =  5.000e-01  max|aij| =  1.933e+03  ratio =  3.866e+03
GM: min|aij| =  1.391e-01  max|aij| =  7.188e+00  ratio =  5.167e+01
EQ: min|aij| =  1.935e-02  max|aij| =  1.000e+00  ratio =  5.167e+01
Constructing initial basis...
Size of triangular part is 23
      0: 

In [15]:
# Solution
if result.solver.termination_condition == pyo.TerminationCondition.optimal:
    print(f'Optimal cost: {pyo.value(model.total_cost)}\n')
    print('Optimal diet:')
    
    for j in model.F:
        if pyo.value(model.x[j]) > 0:
            print(f'  {j}: {pyo.value(model.x[j])}')

Optimal cost: 6.734928786557034

Optimal diet:
  Hamburger: 2.0
  Sweet 'N Sour Sauce: 5.34410702055754
  Honey: 0.655892979442466
  Side Salad: 0.277751220431961
  Cheerios: 1.76541539679489
  Cinnamon Raisin Danish: 0.125755311894635
  Chocolate Shake: 2.34625196212549
  Orange Juice: 0.421489473257796
  H-C Orange Drink (large): 0.232258564616714


__7.__ Create your own minimum cost diet.

__a.__ In the Markdown cell below, in words, describe at least 3 side constraints that modify the original solution from #3 so that it suits conditions you would want in your daily diet. 

*Write your answer here. Double-click to edit.*

b. in the code cell below, write a new model for the diet problem in Pyomo, adding your own side constraints from your answer to #7a. __Start from scratch so that the constraints you added in #5 are not included.__ Solve your new model and output the solution.

In [16]:
# Put your new model here


---

## Grading rubric

| Problem | Points |
| :-: | -: |
| 1 | 10 |
| 2 | 15 |
| 3 | 5 |
| 4 | 10 |
| 5 | 15 |
| 6 | 5 |
| 7a | 10 |
| 7b | 20 |
| __Total__ | __90 points__ |